# Setting up the stage {#sec-20230105101657}

In this chapter, we define the general framework for the analysis of a perfect mechanical system. Notations are defined in @sec-20230208081323. Then, the [SymPy](https://www.sympy.org) library is set up in @sec-20230103084732. Finally, the energy of the system is rewritten in a more convenient form in @sec-20230208081549.

## Mathematical setting {#sec-20230208081323}

The space of admissible states of the system under consideration is denoted $U$. It has the structure of a real vector space. The energy of the system is $\E(u, \lambda)$, where $\lambda$ denotes a loading parameter. It is assumed that the fundamental branch of the equilibrium diagram, $u^\ast(\lambda)$ is known. Then the energy is stationary with respect to the state $u$ along the whole branch. In other words, for all $\hat{u} \in U$
$$
\E_{,u}[u^{\ast}(\lambda), \lambda; \hat{u}]=0,
$$ {#eq-20221227184138}
where $\E_{,u}(u, \lambda; \hat{u})$ denotes the (real) value of the differential of the energy $\E$ with respect to the state $u$, evaluated at $(u, \lambda)$, for the test function $\hat{u}$. Similarly, evaluation of the second-, third-, etc., order differential of the energy will be denoted $\E_{,uu}(u, \lambda; \hat{u}, \hat{v})$, $\E_{,uuu}(u, \lambda; \hat{u}, \hat{v}, \hat{w})$, etc. It is assumed that a finite value $\lambda_0 > 0$ of $\lambda$ can be found (critical load), such that

1. $\E_{,uu}[u(\lambda), \lambda] > 0$ for all $0 < \lambda < \lambda_0$,
2. $\E_{,uu}(u_0, \lambda_0) \geq 0$ but $\E_{,uu}(u_0, \lambda_0) \ngtr 0$,
3. $\E_{,uu}[u(\lambda), \lambda] < 0$ for $\lambda > \lambda_0$, close enough to $\lambda_0$.

The load $\lambda_0$ will be referred to as the *critical load*; similarly, the state $u_0$ of the system at the critical load will be referred to as the *critical state*; finally, the pair $(u_0, \lambda_0)$ is the *critical point* of the system. Assumption 1 implies that equilibria along the fundamental branch are *stable below the critical load*. Conversely, it results from assumption 3 that equilibrium points on the fundamental branch are *unstable above the critical load*. Stability at the critical load is yet undetermined.

The goal of these notes is to analyze *all* equilibrium paths that pass through the critical point $(u_0, \lambda_0)$.

We introduce the following notations
$$
u_0 = u^\ast(\lambda_0), \quad \dot{u}_0 = \frac{\D u^\ast}{\D \lambda} \biggr \rvert_{\lambda=\lambda_0}, \quad \ddot{u}_0 = \frac{\D^2 u^\ast}{\D \lambda^2} \biggr \rvert_{\lambda=\lambda_0}, \quad \dddot{u}_0 = \ldots, \quad \ddddot{u}_0 = \ldots
$$
and
$$
\E_2 = \E_{,uu}(u_0, \lambda_0), \quad \E_3 = \E_{,uuu}(u_0, \lambda_0), \quad \E_4 = \E_{,uuuu}(u_0, \lambda_0).
$$

Note that $\E_2$, $\E_3$ and $\E_4$ thus defined are bi-, tri- and quadrilinear forms, respectively. The following derivatives are also introduced
$$
\dot{\E}_2(\hat{u}, \hat{v}) = \frac{\D}{\D\lambda} \E_{,uu}[ u^\ast(\lambda), \lambda; \hat{u}, \hat{v}] \biggr \rvert_{\lambda = \lambda_0}
= \E_{,uuu}(u_0, \lambda_0; \dot{u}_0, \hat{u}, \hat{v}) + \E_{,uu\lambda}(u_0, \lambda_0; \hat{u}, \hat{v})
$${#eq-20221227184821}
$$
  \begin{aligned}[b]
    \ddot{\E}_2(\hat{u}, \hat{v}) = \frac{\D^2}{\D\lambda^2} \E_{,uu}[ u^\ast(\lambda), \lambda; \hat{u}, \hat{v}] \biggr \rvert_{\lambda = \lambda_0}
	={} & \E_{,uuuu}(u_0, \lambda_0; \dot{u}_0, \dot{u}_0, \hat{u}, \hat{v}) + 2\E_{,uuu\lambda}(u_0, \lambda_0; \dot{u}_0, \hat{u}, \hat{v})\\
    & + \E_{,uu\lambda\lambda}( u_0, \lambda_0; \hat{u}, \hat{v}) + \E_{,uuu}(u_0, \lambda_0, \ddot{u}_0),
  \end{aligned}
$$
and, similarly, $\dot{\E}_3$, $\ddot{\E}_3$, etc.

## Computational setting {#sec-20230103084732}

We will use the [Python](https://www.python.org) library [SymPy](https://www.sympy.org) for symbolic mathematics and rely also on the [IPython](https://ipython.org/) library (in particular, [IPython.display.Math](https://ipython.readthedocs.io/en/stable/api/generated/IPython.display.html#IPython.display.Math)) for pretty $\LaTeX$ output. Some useful functions are defined in the `lsk.py` module, which will be systematically imported.

Without loss of generality, it will be assumed in all symbolic calculations that $\lambda_0 = 0$ and $u_0 = 0$. The general case $\lambda_0 \neq 0$ and $u_0 \neq 0$ is readily recovered through the substitution $\lambda \leftrightarrow \lambda - \lambda_0$ and $u \leftrightarrow u - u_0$.

The following developments involve the energy $(u, \lambda) \mapsto \E(u, \lambda)$ and its differentials at the critical point $(u_0, \lambda_0)$, as well as the fundamental path $\lambda \mapsto u^\ast(\lambda)$ and its derivatives at $\lambda = \lambda_0$. It will therefore be convenient to express $\E$ and $u^\star$ as Taylor expansions with respect to $u$ and $\lambda$.

We start with the Taylor expansion of the energy $\E$. We define its differentials at the critical point. These differentials are stored in a dictionary. Values are indexed with the order of the differentials with respect to $u$ and $\lambda$.

In the following code blocks, `n` is the total order of the differential, `p` is the order of the differential with respect to $u$ and `q = n - p` is the order of the differential with respect to $\lambda$.

In [1]:
from lsk import *

λ, u, v, u_hat, v_hat, w_hat = sympy.symbols(r"\lambda u v \hat{u} \hat{v} \hat{w}")

In [2]:
dE = dict()
for n in range(1, 5):
    for p in range(n + 1):
        q = n - p
        if q == 0:
            dE[p, q] = sympy.Symbol(r"\E_{}".format(p))
        else:
            dE[p, q] = sympy.Symbol(r"\E_{" + p * "u" + q * "\lambda" + "}")
            
# Remember that the energy is stationary at the critical point (equilibrium)
dE[1, 0] = zero

The dictionary `dE` should be understood as follows: `dE[(2, 1)] == dE[2, 1]` is the symbol $\E_{uu\lambda}$, which stands for $\E_{,uu\lambda}(u_0, \lambda_0; \bullet, \bullet)$. The following symbols have been defined.

In [3]:
#| code-fold: true

_dict = dict()
for n in range(1, 5):
    for p in range(n + 1):
        q = n - p
        _dict[dE[p, q]] = ("\\E_{," + p * "u" 
                           + q * "\\lambda"
                           + "}(u_0, \lambda_0)")
        
display_latex_dict(_dict, num_cols=3)

<IPython.core.display.Math object>

where

- $\E_\lambda$, $\E_{\lambda\lambda}$, $\E_{\lambda\lambda\lambda}$ and $\E_{\lambda\lambda\lambda\lambda}$ are *scalar quantities*,
- $\E_{u\lambda}$, $\E_{u\lambda\lambda}$ and $\E_{u\lambda\lambda\lambda}$ are *linear forms*,
- $\E_2$, $\E_{uu\lambda}$ and $\E_{uu\lambda\lambda}$ are *bilinear forms*,
- $\E_3$ and $\E_{uuu\lambda}$ are *trilinear forms*,
- $\E_4$  is a *quadrilinear form*.

Note that all these differentials are defined as SymPy *scalars*. A definition as a SymPy *function* (e.g. `E2 = Function(r"\E_2")`) would be more appropriate. However, SymPy would fail to account for multilinearity or symmetry of these forms. Therefore, we use the following trick: all multi-linear forms are defined as scalars, and the standard multiplication operator `*` means function application. In other words, `E2 * (α * u + β * v) * w` (resp. `E2 * u * v - E2 * v * u`) should be understood as `E2(α * u + β * v, w)` (resp. `E2(u, v) - E2(v, u)`). In both cases, the expressions would are correctly simplified.

Whether the symbols in an expression are true scalars or vectors (elements of $U$) should be clear from the context. For example, in the expression: `λ * E2 * u * v`, the first `*` is a true multiplication, while the other `*` refer to function application.

The energy $\E(u, \lambda)$ is now expressed as a Taylor expansion about the critical point.

In [4]:
_f = lambda p, q: sympy.binomial(p + q, p) / sympy.factorial(p + q)
E = sum(_f(p, q) * dE_pq * u**p * λ**q for (p, q), dE_pq in dE.items())

In [5]:
#| code-fold: true
display_latex_equation(r"\E(u, \lambda)", E)

<IPython.core.display.Math object>

We now turn to the definition of the fundamental path $\lambda \mapsto u^\star(\lambda)$. We first define in a dictionary the derivatives of $u^\star$ with respect to $\lambda$, at $\lambda = \lambda_0 = 0$. For example, `du_0[2]` is the symbol $\ddot{u}_0$, which stands for $\D^2 u^\star / \D \lambda^2 \rvert_{\lambda=\lambda_0}$.

In [6]:
du_0 = {n: sympy.Symbol("\\" + "".join(n * ("d",)) + "ot{u}_0") for n in range(1, 5)}

_dict = {f"\\frac{{\\D^{k}u^\\ast}}{{\\D \\lambda^{k}}}" : v for k, v in du_0.items()}

In [7]:
#| code-fold: true
display_latex_dict(_dict, num_cols=4)

<IPython.core.display.Math object>

The fundamental path is then defined through its Taylor expansion.

In [8]:
u_star = sum(λ**n / sympy.factorial(n) * v for n, v in du_0.items())

In [9]:
#| code-fold: true
display_latex_equation(r"u^\star(\lambda)", u_star)

<IPython.core.display.Math object>

It will be assumed in the following that $v, \hat{v} \in V$, therefore terms of the form $\E_2(\bullet, v)$ and $\E_2(\bullet, \hat{v})$ should vanish. The following function performs this simplification.

In [10]:
# Let's define a convenient alias to the Hessian
E2 = dE[2, 0]

def simplify_E2(expr):
    expr = expr.expand()
    coeff_E2 = expr.coeff(E2).expand()
    coeff_E2_v = coeff_E2.coeff(v)
    coeff_E2_v_hat = (coeff_E2 - coeff_E2_v * v).expand().coeff(v_hat)
    return (expr - (coeff_E2_v * E2 * v + coeff_E2_v_hat * E2 * v_hat)).expand()

assert simplify_E2(E2 * v) == 0
assert simplify_E2(E2 * v_hat) == 0
assert simplify_E2(3 * v + 4 * v_hat + 5 * v * v_hat + 6 * E2 * v + 7 * E2 * v_hat + 8 * E2 * v * v_hat) == 3 * v + 4 * v_hat + 5 * v * v_hat

## Elimination of the mixed derivatives of the energy {#sec-20230208081549}

Since the fundamental path $\lambda \mapsto u^\ast(\lambda)$ is an equilibrium path, the various differentials of the energy at the critical point are not linearly independent. To express the relationships between these forms, we define $\mathcal R^\ast(\lambda; \bullet)$ as the jacobian of the energy along the fundamental path $u^\ast(\lambda)$
$$
\mathcal R^\ast(\lambda; \bullet) = \E_{,u}[u^\ast(\lambda), \lambda; \bullet].
$$

In [11]:
R_star = (E.diff(u) * u_hat).subs(u, u_star).expand()

In [12]:
#| code-fold: true
display_latex_equation(r"\mathcal{R}^\ast(\lambda;" + sympy.latex(u_hat) + ")",
                       R_star.series(λ, 0, 5))

<IPython.core.display.Math object>

Of course, since $\lambda \mapsto u^\ast(\lambda)$ is an equilibrium path, we have $\mathcal R^\ast(\lambda; \bullet) = 0$ for all $\lambda$. Therefore, all coefficients of the above polynomial in $\lambda$ are null, which delivers expressions of $\E_{u\lambda}$, $\E_{u\lambda\lambda}$ and $\E_{u\lambda\lambda\lambda}$

In [13]:
mixed1 = dict()

for q in range(1, 4):
    x = dE[1, q]
    eq = R_star.coeff(λ, q)
    sol = sympy.solve(eq, x)[0]
    mixed1[x] = sol

In [14]:
#| code-fold: true
display_latex_dict(mixed1, num_cols=1)

<IPython.core.display.Math object>

We want to get rid of the remaining mixed derivatives, namely: $\E_{uu\lambda}$, $\E_{uuu\lambda}$ and $\E_{uu\lambda\lambda}$. To do so, we introduce the derivatives $\dot{\E}_2$, $\ddot{\E}_2$ and $\dot{\E}_3$ defined in @sec-20230105101657.

In [15]:
E2_dot, E2_ddot, E3_dot = sympy.symbols("\dot{\E}_2 \ddot{\E}_2 \dot{\E}_3")

E_uu_star = E.diff(u, 2).subs(u, u_star).expand()
E_uuu_star = E.diff(u, 3).subs(u, u_star).expand()

eqs = [sympy.Eq(E2_dot, E_uu_star.coeff(λ, 1)),
       sympy.Eq(E3_dot, E_uuu_star.coeff(λ, 1)),
       sympy.Eq(E2_ddot, 2 * E_uu_star.coeff(λ, 2))]

unknowns = [dE[2, 1], dE[3, 1], dE[2, 2]] # E_uuλ, E_uuuλ, E_uuλλ

mixed2 = dict()
for eq, x in zip(eqs, unknowns):
    mixed2[x] = sympy.solve(eq, x)[0].subs(mixed2).expand()

We get the following expressions

In [16]:
#| code-fold: true
display_latex_dict(mixed2, num_cols=1)

<IPython.core.display.Math object>

Combining the above results allows to fully eliminate the mixed derivatives

In [17]:
#| code-fold: true

mixed = {k: v.subs(mixed2).expand() for k, v in mixed1.items()}
mixed.update(mixed2)

display_latex_dict(mixed, num_cols=1)

<IPython.core.display.Math object>

These expressions can be plugged into the expansion of the energy.

In [18]:
E = E.subs(mixed).expand()

In [19]:
#| code-fold: true
display_latex_equation(r"\E(u, \lambda)", E)

<IPython.core.display.Math object>

Some of the previous symbolic results are now saved for further use in the subsequent notebooks.

In [20]:
data = {"E" : sympy.srepr(E)}

with open("problem_setting.out.json", "w") as f:
    json.dump(data, f)